In [ ]:
import h5py
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pint import UnitRegistry
import os
ureg = UnitRegistry();
plt.rcParams.update({'font.size': 20})

In [ ]:
def get_runtimes(filename):
    fh = h5py.File(filename,'r')
    timer_string = fh['timer/rank0'][()].decode('UTF-8')
    timer_string_lines = timer_string.split('\n')
    data = dict()
    for line in timer_string_lines[7:-1]:
        label = line.split("  ")[0].strip()
        value_string = line.split()[-6].replace('μs', 'microseconds')
        value = ureg(value_string).to(ureg.s).magnitude
        data[label] = value
    fh.close()
    return data

In [ ]:
results = {}
ranks = np.logspace(0,6,num=7,base=2,dtype=int)
for r in ranks:
    filename = f"weak_scaling_r{r}.h5"
    results[r] = get_runtimes(filename)
df = pd.DataFrame(results)
fields = ['Particle State Update', 
          'Resample',
          'State Copy',
          'Process Noise', 
          'Particle Variance',
          'Particle Mean',
          'IO',
          'MPI Scatter',
          'MPI Gather']
df

# Weak Scaling -- Scaling to larger problem size
## Constant run time = Perfect scaling
* 64 particles per MPI rank
* 1 thread per MPI rank
* 1 Node on CSD3-KNL

In [ ]:
ax = df.transpose().plot(y = fields, loglog = True, xticks = ranks, figsize = (16,9), style = '-o')
ax.set_xlabel("Number of Ranks");
ax.set_ylabel("Run Time (s)");
ax.set_xticklabels(ranks);

In [ ]:
results = {}
ranks = np.logspace(0,4,num=5,base=2,dtype=int)
for r in ranks:
    filename = f"/Users/tkoskela/csd3/tdac/scaling_test_5jun20/weak_scaling/weak_scaling_threads_r{r}.h5"
    results[r] = get_runtimes(filename)
df = pd.DataFrame(results)
df

# Weak Scaling
 * 256 particles per MPI rank
 * 4 threads per MPI rank
 * 1 node on CSD3-KNL

In [ ]:
ax = df.transpose().plot(y = fields, loglog = True, xticks = ranks, figsize = (16,9), style = '-o',)
ax.set_xlabel("Number of Ranks");
ax.set_ylabel("Run Time (s)");
ax.set_xticklabels(ranks);

In [ ]:
results = {}
ranks = np.logspace(0,6,num=7,base=2,dtype=int)
for r in ranks:
    filename = f"/Users/tkoskela/csd3/tdac/scaling_test_5jun20/weak_scaling/weak_scaling_multinode_r{r}.h5"
    results[r] = get_runtimes(filename)
df = pd.DataFrame(results)
df

# Weak Scaling
 * 256 particles per MPI rank
 * 4 threads per MPI rank
 * 4 nodes on CSD3-KNL (Default pinning)

In [ ]:
ax = df.transpose().plot(y = fields, loglog = True, xticks = ranks, figsize = (16,9), style = '-o',)
ax.set_xlabel("Number of Ranks");
ax.set_ylabel("Run Time (s)");
ax.set_xticklabels(ranks);

In [ ]:
filename = "/Users/tkoskela/csd3/tdac/scaling_test_5jun20/weak_scaling/weak_scaling_r1.h5"
fh = h5py.File(filename, "r")
params = fh["params"].attrs
for p in list(params):
    try:
        print(p, params[p])
    except TypeError:
        pass    